In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [2]:
import re
import base64
import json
import numpy as np
import pandas as pd

In [3]:
# Use a service account
cred = credentials.Certificate('key/oduFireKey.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

# Routes

In [4]:
routes_ref = db.collection(u'Routes')
routes_firebaseGenerator = routes_ref.stream()
routes = [doc.to_dict() for doc in routes_firebaseGenerator]

In [5]:
routes[0]

{'from_location': 'Thodupuzha',
 'to_location': 'Moolamattom',
 'bus_code': 10000,
 'via_route': 'Muttom',
 'bus_type': 'Ordinary',
 'route_name': 'Thodupuzha - Moolamattom'}

In [30]:
route_id = 1000000
route_dict = {}
_id = 'a'
for route in routes:
    routeName = [route['from_location'],' - ',route['to_location']]
    opt_1 = ''.join(routeName)
    opt_1 = opt_1 # + ', via_route: '+ route['via_route']
    opt_2 = ''.join(routeName[::-1]) 
    opt_2 = opt_2 # + ', via_route: '+ route['via_route']
    opt_1, opt_2 = opt_1.strip(), opt_2.strip()
    flag_1, flag_2 = 0, 0
    if(opt_1 in route_dict):
        flag_1 = 1
    if(opt_2 in route_dict):
        flag_2 = 1
    if(flag_1 == 0 and flag_2 == 0):
        if(len(route['via_route'])!=0):
            route_dict[opt_1] = {
                'id':str(route_id)[1:],
                'route_name':route['route_name'],
                'via':1,
                'via_route':{route['via_route']:_id}
            }
        else:
            route_dict[opt_1] = {
                'id':str(route_id)[1:],
                'route_name':route['route_name'],
                'via':0,
                'via_route':{}
            }
        route_id+=1
    else:
        if(flag_1 == 1):
            if(route['via_route'] not in route_dict[opt_1]['via_route']):
                route_dict[opt_1]['via_route'][route['via_route']] = chr(ord(_id) + route_dict[opt_1]['via'])
                route_dict[opt_1]['via']+=1
        else:
            if(route['via_route'] not in route_dict[opt_2]['via_route']):
                route_dict[opt_2]['via_route'][route['via_route']] = chr(ord(_id) + route_dict[opt_2]['via'])
                route_dict[opt_2]['via']+=1

In [31]:
len(route_dict)

6105

In [32]:
routes_dict = {value['id']:value for key,value in route_dict.items()}
via_routes = [{y:x for x,y in value['via_route'].items()} for key,value in routes_dict.items()]
via_routes = [{key:[x.strip() for x in value.split(',')] for key,value in via_r.items()} for via_r in via_routes]
i = 0
for key, value in routes_dict.items():
    routes_dict[key]['via_route'] = via_routes[i]
    i+=1
via_routes[:3]

[{'a': ['Muttom'],
  'b': ['Muttom', 'Kakkomb'],
  'c': ['Velliyamattom', 'Poochapra'],
  'd': ['Kakkomb'],
  'e': ['Velliyavattom', 'Guruthikalam'],
  'f': ['Kanjar', 'Muttom'],
  'g': ['Muttom', 'Kanjar']},
 {'a': ['Moolamattom', 'Cheruthoni', 'Karimban', 'Murikkassery'],
  'b': ['Murikkassery', 'Cheruthoni', 'Moolamattom'],
  'c': ['Murikkassery', 'Cheruthoni', 'Uppukunnu'],
  'd': ['Uppukunnu', 'Cheruthoni', 'Karimban', 'Murikkassery']},
 {'a': ['Murikkassery', 'Cheruthoni', 'Moolamattom', 'Thodupuzha', 'Pala']}]

In [33]:
[x for x in routes_dict.values()][:3]

[{'id': '000000',
  'route_name': 'Thodupuzha - Moolamattom',
  'via': 7,
  'via_route': {'a': ['Muttom'],
   'b': ['Muttom', 'Kakkomb'],
   'c': ['Velliyamattom', 'Poochapra'],
   'd': ['Kakkomb'],
   'e': ['Velliyavattom', 'Guruthikalam'],
   'f': ['Kanjar', 'Muttom'],
   'g': ['Muttom', 'Kanjar']}},
 {'id': '000001',
  'route_name': 'Thodupuzha - Thopramkudy',
  'via': 4,
  'via_route': {'a': ['Moolamattom', 'Cheruthoni', 'Karimban', 'Murikkassery'],
   'b': ['Murikkassery', 'Cheruthoni', 'Moolamattom'],
   'c': ['Murikkassery', 'Cheruthoni', 'Uppukunnu'],
   'd': ['Uppukunnu', 'Cheruthoni', 'Karimban', 'Murikkassery']}},
 {'id': '000002',
  'route_name': 'Thopramkudy - Kottayam',
  'via': 1,
  'via_route': {'a': ['Murikkassery',
    'Cheruthoni',
    'Moolamattom',
    'Thodupuzha',
    'Pala']}}]

In [37]:
routes_dict['000536']

{'id': '000536',
 'route_name': 'Thrissur - Piravom',
 'via': 1,
 'via_route': {'a': ['Chalakudy', 'Ernakulam', 'Chottanikara']}}

In [34]:
with open('routes.txt', 'w') as routes_file:
     routes_file.write(json.dumps(routes_dict))

### Upload collection to firebase collection 
#### "Unique Routes" Collection

In [65]:
uniqueRoutes_ref = db.collection(u'Unique Routes')

In [ ]:
transaction = db.transaction()
uniqueRoutes_ref = db.collection(u'Unique Routes')

@firestore.transactional
def update_in_transaction(transaction, ref):
#     snapshot = ref.get(transaction=transaction)
    temp_1 = {
        'id':value['id'],
        'route_name':value['route_name']
    }
    transaction.set(ref, )

update_in_transaction(transaction, uniqueRoutes_ref)

In [67]:
for key, value in routes_dict.items():
    temp_1 = {
        'id':value['id'],
        'route_name':value['route_name']
    }
    uniqueRoutes_ref.document(value['id']).set(temp_1)
    for via_key,via_value in value['via_route'].items():
        temp_2 = {via_key:via_value}
        uniqueRoutes_ref.document(value['id']).collection('Via Route(s)').document(via_key).set(temp_2)

# Bookings

In [ ]:
bookings_ref = db.collection(u'Bookings')
bookings_firebaseGenerator = bookings_ref.stream()
bookings = [doc.to_dict() for doc in bookings_firebaseGenerator]

In [ ]:
bookings[0]

# Stops

In [ ]:
stops_ref = db.collection(u'Bus Stops')
stops_firebaseGenerator = stops_ref.stream()
stops = [doc.to_dict() for doc in stops_firebaseGenerator]

In [ ]:
stops_df.loc[:10,["Stop Name"]]

In [ ]:
pattern = r"([\w]+)[ [\w]+]?"

In [ ]:
bool(re.match("ERNAK",stops[0]["Stop Name"]))

# Bus Static Locations

In [48]:
staticLoc_ref = db.collection(u'Bus Static Locations')
staticLoc_firebaseGenerator = staticLoc_ref.stream()
staticLoc = [doc.to_dict() for doc in staticLoc_firebaseGenerator]

In [49]:
staticLoc[0]

{'Direction': 'DOWN',
 'Location': <google.cloud.firestore_v1._helpers.GeoPoint at 0x1a111ecf6d0>,
 'Latitude': '9.935224849699814',
 'Longitude': '76.30169481075814',
 'Bus_ID': 'Bus 0',
 'Route_ID': '000895',
 'count': 37,
 'Route_Name': 'Ernakulam BS - Pala'}

In [50]:
directions = ['UP','DOWN']

In [51]:
firestore.GeoPoint('1','1')

In [52]:
busId = ['Bus '+str(x) for x in range(16)]
direction = [directions[x] for x in np.random.randint(low = 0, high = 2,size = 16)]
location = [
    firestore.GeoPoint(9.935224849699814, 76.30169481075814),
    firestore.GeoPoint(9.879083142900397, 76.3779124578617),
    firestore.GeoPoint(9.810753207094008, 76.44795029574063),
    firestore.GeoPoint(9.74037894176729, 76.61755172668276),
    firestore.GeoPoint(9.992716038015864, 76.30587231478111),
    firestore.GeoPoint(10.014692735997073, 76.31136547853633),
    firestore.GeoPoint(10.036329876643578, 76.31376873767923),
    firestore.GeoPoint(10.092444463411299, 76.34569775200639),
    firestore.GeoPoint(9.322197530861805, 76.63164818968484),
    firestore.GeoPoint(9.332708618220876, 76.66833815329281),
    firestore.GeoPoint(9.311686126956236, 76.72430302865796),
    firestore.GeoPoint(9.274142819888679, 76.76572718111854),
    firestore.GeoPoint(10.449444584801288, 76.26452260181959),
    firestore.GeoPoint(10.275179509451979, 76.35241322190298),
    firestore.GeoPoint(10.098114272912156, 76.48974231578322),
    firestore.GeoPoint(9.966941978331834, 76.60509875464265)    
]
routeId = ['000895']*4+['004139']*4+['004600']*4+['000086']*4
count = [int(x) for x in np.random.randint(low = 20, high = 65,size=16)]
RouteName = ['Ernakulam - Pala']*4+['Aluva - Ernakulam']*4+['Chengannur - Pathanamthitta']*4+['Thrissur - Thodupuzha']*4

In [53]:
latitude = [str(x.latitude) for x in location]
longitude = [str(x.longitude) for x in location]

In [54]:
data = [
  {
      'Bus_ID': busId[0],
      'Direction': direction[0], 
      'Latitude': latitude[0],
      'Location': location[0],
      'Longitude': longitude[0],
      'Route_ID': routeId[0],
      'count': count[0],
      'Route_Name': RouteName[0]
  },
  {
      'Bus_ID': busId[1],
      'Direction': direction[1], 
      'Latitude': latitude[1],
      'Location': location[1],
      'Longitude': longitude[1],
      'Route_ID': routeId[1],
      'count': count[1],
      'Route_Name': RouteName[1]
  },
  {
      'Bus_ID': busId[2],
      'Direction': direction[2], 
      'Latitude': latitude[2],
      'Location': location[2],
      'Longitude': longitude[2],
      'Route_ID': routeId[2],
      'count': count[2],
      'Route_Name': RouteName[2]
  },
  {
      'Bus_ID': busId[3],
      'Direction': direction[3], 
      'Latitude': latitude[3],
      'Location': location[3],
      'Longitude': longitude[3],
      'Route_ID': routeId[3],
      'count': count[3],
      'Route_Name': RouteName[3]
  },
  {
      'Bus_ID': busId[4],
      'Direction': direction[4], 
      'Latitude': latitude[4],
      'Location': location[4],
      'Longitude': longitude[4],
      'Route_ID': routeId[4],
      'count': count[4],
      'Route_Name': RouteName[4]
  },
  {
      'Bus_ID': busId[5],
      'Direction': direction[5], 
      'Latitude': latitude[5],
      'Location': location[5],
      'Longitude': longitude[5],
      'Route_ID': routeId[5],
      'count': count[5],
      'Route_Name': RouteName[5]
  },
  {
      'Bus_ID': busId[6],
      'Direction': direction[6], 
      'Latitude': latitude[6],
      'Location': location[6],
      'Longitude': longitude[6],
      'Route_ID': routeId[6],
      'count': count[6],
      'Route_Name': RouteName
  },
  {
      'Bus_ID': busId[7],
      'Direction': direction[7], 
      'Latitude': latitude[7],
      'Location': location[7],
      'Longitude': longitude[7],
      'Route_ID': routeId[7],
      'count': count[7],
      'Route_Name': RouteName[7]
  },
  {
      'Bus_ID': busId[8],
      'Direction': direction[8], 
      'Latitude': latitude[8],
      'Location': location[8],
      'Longitude': longitude[8],
      'Route_ID': routeId[8],
      'count': count[8],
      'Route_Name': RouteName[8]
  },
  {
      'Bus_ID': busId[9],
      'Direction': direction[9], 
      'Latitude': latitude[9],
      'Location': location[9],
      'Longitude': longitude[9],
      'Route_ID': routeId[9],
      'count': count[9],
      'Route_Name': RouteName[9]
  },
  {
      'Bus_ID': busId[10],
      'Direction': direction[10], 
      'Latitude': latitude[10],
      'Location': location[10],
      'Longitude': longitude[10],
      'Route_ID': routeId[10],
      'count': count[10],
      'Route_Name': RouteName[10]
  },
  {
      'Bus_ID': busId[11],
      'Direction': direction[11], 
      'Latitude': latitude[11],
      'Location': location[11],
      'Longitude': longitude[11],
      'Route_ID': routeId[11],
      'count': count[11],
      'Route_Name': RouteName[11]
  },
  {
      'Bus_ID': busId[12],
      'Direction': direction[12], 
      'Latitude': latitude[12],
      'Location': location[12],
      'Longitude': longitude[12],
      'Route_ID': routeId[12],
      'count': count[12],
      'Route_Name': RouteName[12]
  },
  {
      'Bus_ID': busId[13],
      'Direction': direction[13], 
      'Latitude': latitude[13],
      'Location': location[13],
      'Longitude': longitude[13],
      'Route_ID': routeId[13],
      'count': count[13],
      'Route_Name': RouteName[13]
  },
  {
      'Bus_ID': busId[14],
      'Direction': direction[14], 
      'Latitude': latitude[14],
      'Location': location[14],
      'Longitude': longitude[14],
      'Route_ID': routeId[14],
      'count': count[14],
      'Route_Name': RouteName[14]
  },
  {
      'Bus_ID': busId[15],
      'Direction': direction[15], 
      'Latitude': latitude[15],
      'Location': location[15],
      'Longitude': longitude[15],
      'Route_ID': routeId[15],
      'count': count[15],
      'Route_Name': RouteName[15]
  }
]


In [55]:
for _id in range(16):
    staticLoc_ref.document('Bus '+str(_id)).set(data[_id])

# Schedule Suggest

In [3]:
data = pd.DataFrame(
    {'Route':
     {0: 'ponkunnam kozhikkode', 
      1: 'muvattupuzha kumily', 
      2: 'kumily ernakulam', 
      3: 'kollam oachira', 
      4: 'oachira kollam'
     }, 
     'Exceed_Diminish':
     {0: 1, 1: 0, 2: 0, 3: 0, 4: 0},
     'Count_exceeded_diminished':
     {0: 21.0, 1: 518.0, 2: 2638.0, 3: 758.0, 4: 1879.0}
    }
)

In [10]:
def standardizeRoutes(row):
    datList = row.split(' ')
    datList[0] = datList[0][0].upper()+datList[0][1:].lower()
    datList[1] = datList[1][0].upper()+datList[1][1:].lower()
    return " - ".join(datList)

In [11]:
data = data.rename({'Exceed_Diminish':'status','Count_exceeded_diminished':'count'},axis=1)
data['status'] = data['status'].replace({1:'Surplus',0:'Shortage'})
data['Route'] = data['Route'].apply(standardizeRoutes)
# data

0    Ponkunnam - Kozhikkode
1     Muvattupuzha - Kumily
2        Kumily - Ernakulam
3          Kollam - Oachira
4          Oachira - Kollam
Name: Route, dtype: object

In [9]:
suggestSchd_ref = db.collection(u'Schedule Suggest')

In [10]:
def upload_collection(data):
    for i in range(data.shape[0]):
        tmp = {
            'routeName':data.loc[i,'Route'],
            'status':data.loc[i,'status'],
            'count':data.loc[i,'count']
        }
        suggestSchd_ref.document().set(tmp)
    return 'Done'

In [11]:
upload_collection(data)

'Done'